In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, transforms
import numpy as np
import random
from torchvision.utils import save_image
import os
from torch.optim.lr_scheduler import StepLR
import glob

In [ ]:
if (os.path.exists("./output")) == False:
    os.mkdir("output")

if (os.path.exists("./model_weight")) == False:
    os.mkdir("model_weight")

for epoch in range (200):
    if (os.path.exists("./output/%03d" % epoch)) == False:
        os.mkdir("./output/%03d" % epoch)
    else:
        files = glob.glob("./output/%03d/*.png" % epoch)

        for f in files:
          os.remove(f)

In [ ]:
BATCH_SIZE = 256
EPOCHS = 20 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,)),
                           transforms.Resize(32)
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,)),
                           transforms.Resize(32)
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
def plotImg(img_array, reconstructed_array = None):
    fig, axs = plt.subplots(2, 5, figsize=(15, 6))
    for i in range (10):
        random_index = random.randint(0, BATCH_SIZE)
        random_image = img_array[random_index]
        if reconstructed_array:
            pass
        else:
            axs[i // 5][i % 5].imshow(np.transpose(random_image, (1, 2, 0)))
        
    plt.show()

In [ ]:
dataset_iter = iter(train_loader)

img, label = dataset_iter.next()
plotImg(img)

In [ ]:
class Block(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size = 3)
        self.relu  = nn.ReLU()
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size = 3)
    
    def forward(self, x):
        return self.relu(self.conv2(self.relu(self.conv1(x))))

class Encoder(nn.Module):
    def __init__(self, chs=(1, 64, 128)):
        super().__init__()
        self.enc_blocks = nn.ModuleList([Block(chs[i], chs[i+1]) for i in range(len(chs)-1)])
        self.pool       = nn.MaxPool2d(2)
    
    def forward(self, x):
        ftrs = []
        for block in self.enc_blocks:
            x = block(x)
            ftrs.append(x)
            x = self.pool(x)
        return ftrs

class Decoder(nn.Module):
    def __init__(self, chs=(128, 64)):
        super().__init__()
        self.chs         = chs
        self.upconvs    = nn.ModuleList([nn.ConvTranspose2d(chs[i], chs[i+1], 2, 2) for i in range(len(chs)-1)])
        self.dec_blocks = nn.ModuleList([Block(chs[i], chs[i+1]) for i in range(len(chs)-1)]) 
        
    def forward(self, x, encoder_features):
        for i in range(len(self.chs)-1):
            x        = self.upconvs[i](x)
            enc_ftrs = self.crop(encoder_features[i], x)
            x        = torch.cat([x, enc_ftrs], dim=1)
            x        = self.dec_blocks[i](x)
        return x
    
    def crop(self, enc_ftrs, x):
        _, _, H, W = x.shape
        enc_ftrs   = torchvision.transforms.CenterCrop([H, W])(enc_ftrs)
        return enc_ftrs

class UNet(nn.Module):
    def __init__(self, enc_chs=(1, 64, 128), dec_chs=(128, 64), 
                    num_class=1, retain_dim=True, out_sz=(32,32)):
        super().__init__()
        self.encoder     = Encoder(enc_chs)
        self.decoder     = Decoder(dec_chs)
        self.head        = nn.Conv2d(dec_chs[-1], num_class, 1)
        self.retain_dim  = retain_dim
        self.out_sz      = out_sz

    def forward(self, x):
        enc_ftrs = self.encoder(x)
        out      = self.decoder(enc_ftrs[::-1][0], enc_ftrs[::-1][1:])
        out      = self.head(out)
        if self.retain_dim:
            out = F.interpolate(out, self.out_sz)
        return out

In [ ]:
def training(model, train_loader, criterion, optimizer, scheduler,  use_cuda = False):
    dataset_iter = iter(train_loader)
    len_dataloader = len(dataset_iter)

    i = 0
    while i < len_dataloader:
        org, label = dataset_iter.next()

        optimizer.zero_grad()

        recon = model(org)

        loss = criterion(recon, org)

        loss.backward()
        optimizer.step()

        if  i % 200 == 0:
            real = org.data
            reconstruction= recon.data

            save_image(real, './output/%03d/%d_A.png' % ( epoch, i))
            save_image(reconstruction, 'output/%03d/%d_reconA.png' % ( epoch, i))
        
        i += 1

        scheduler.step()


In [ ]:
unet = UNet()

learning_rate = 0.02
beta = (0.5, 0.999)

criterion_adv = torch.nn.MSELoss()
criterion_construct = torch.nn.L1Loss()

optimizer = torch.optim.Adam(unet.parameters(), lr=learning_rate, betas=beta)

scheduler = StepLR(optimizer, step_size=10, gamma=0.8)

In [ ]:
training(unet, train_loader, criterion_adv, optimizer, scheduler)